This code uses Safegraph socioeconomic data, selects relevant columns, and saves.

In [ ]:
import pandas as pd
import glob
import os
from tqdm import tqdm

In [ ]:
root = ''
path = root + 'Data/Safegraph_CENSUS/'
result_path = root + 'final_data/'

# Combine all csvs for full dataset

In [ ]:
# Assemble the list of all the CBG in the dataset
csv_files = glob.glob(os.path.join(path, "*.csv"))
list_cbg = []

for csv in csv_files:
    file = csv.split('\\')[-1]
    if len(file) == 16:
        list_cbg += [file]

In [ ]:
# Sort the list
list_cbg = sorted(list_cbg)

In [ ]:
# Compile socioenonomic data into one dataframe
df_cbg = pd.DataFrame()
for file in tqdm(list_cbg):
    df = pd.read_csv(path + file,index_col=[0])
    try:
        df_cbg = df_cbg.append(df)
    except:
        df_cbg = df.copy()

# Selected columns

In [ ]:
# Read variables of interest
# Full list available in Safegraph's 2020_METADATA_CBG_FIELD_DESCRIPTIONS.csv
selection = 'selected_cols_EV.csv' # Manually prepared by authors
df_cols = pd.read_csv(selection,index_col=[0])
df_cols.tail()

In [ ]:
# Identify the table number
df_cols['TABLE_NUMBER_csv'] = df_cols['TABLE_NUMBER'].str[:3]

In [ ]:
# Compile socioeconomic data into one dataframe
df_socioecon = pd.DataFrame()
for table in tqdm(sorted(set(df_cols['TABLE_NUMBER_csv']))):
    df_table = pd.read_csv(path + '2020_CBG_' + table + '.csv',index_col=[0])
    cols = df_cols.loc[df_cols['TABLE_NUMBER_csv'] == table].index.to_list()
    df_table = df_table[cols]
    try:
        df_socioecon = df_socioecon.merge(df_table,left_index=True,right_index=True,how='outer')
    except:
        df_socioecon = df_table.copy()

In [ ]:
# Rename columns as specified in df_cols / selected_cols_EV.csv
for ind in df_cols.index:
    df_socioecon.rename(columns={ind:df_cols['name'].loc[ind]},inplace=True)

In [ ]:
# Create STATEFIPS for state inspection
df_socioecon['STATEFIPS'] = df_socioecon.index
df_socioecon['STATEFIPS'] = df_socioecon['STATEFIPS'].astype(str).str.zfill(12).str[:2]

In [ ]:
# Normalize: By population
col_bypop = ['total_mixedrace',
       'total_white', 'total_black', 'total_native', 'total_asian',
       'total_workoutofhome']
for col in col_bypop:
	df_socioecon['share_'+col.split('_',1)[1]] = df_socioecon[col]/df_socioecon['total_pop']
	df_socioecon.drop(col,axis=1,inplace=True)

In [ ]:
# Normalize: By commutes
col_bypop = ['total_mode_public', 'total_mode_taxi', 'total_mode_motorcycle',
       'total_mode_bike', 'total_mode_walk', 'total_mode_car',
       'total_mode_other', 'total_workfromhome', 'total_mode_caralone']
for col in col_bypop:
	df_socioecon['share_'+col.split('_',1)[1]] = df_socioecon[col]/df_socioecon['total_modes']
	df_socioecon.drop(col,axis=1,inplace=True)

In [ ]:
# Normalize: Average commute time
df_socioecon['avg_commutetime'] = df_socioecon['agg_commutetime']/df_socioecon['total_modes']
df_socioecon.drop('agg_commutetime',axis=1,inplace=True)

In [ ]:
# Normalize: By commuting times
col_bypop = ['total_commute40', 'total_commute45', 'total_commute60',
       'total_commute90', 'total_commute0', 'total_commute5',
       'total_commute10', 'total_commute15', 'total_commute20',
       'total_commute25', 'total_commute30', 'total_commute35']
for col in col_bypop:
	df_socioecon['share_'+col.split('_',1)[1]] = df_socioecon[col]/df_socioecon['total_modes']
	df_socioecon.drop(col,axis=1,inplace=True)

In [ ]:
# Normalize: Children + by education
col_bypop = ['total_children', 'total_education', 'total_highschool',
       'total_noschooling', 'total_college', 'total_associate',
       'total_bachelor', 'total_master', 'total_professional', 'total_PhD',
       'total_bachelorbyfield', 'total_business', 'total_CS', 'total_environ',
       'total_physical', 'total_engineering', 'total_engineeringother']
for col in col_bypop:
	df_socioecon['share_'+col.split('_',1)[1]] = df_socioecon[col]/df_socioecon['total_modes']
	df_socioecon.drop(col,axis=1,inplace=True)

In [ ]:
# Normalize: Housing density
df_socioecon['housingunits_percapita'] = df_socioecon['total_housingunits']/df_socioecon['total_pop']

In [ ]:
# Normalize: Housing types
col_bypop = ['total_housing_occupied', 'total_housing_vacant',
       'total_housing_owners', 'total_housing_renters',
       'total_housing_detached', 'total_housing_atached',
       'total_housing_heating_solar']
for col in col_bypop:
	df_socioecon['share_'+col.split('_',1)[1]] = df_socioecon[col]/df_socioecon['total_housingunits']
	df_socioecon.drop(col,axis=1,inplace=True)

In [ ]:
# Include additional variables
df_socioecon['share_nonwhite'] = 1. - df_socioecon['share_white']
df_socioecon['share_mode_walkbike'] = df_socioecon['share_mode_walk'] + df_socioecon['share_mode_bike']
df_socioecon['short_commute_0to25min'] = df_socioecon['share_commute0'] + df_socioecon['share_commute5'] + df_socioecon['share_commute10'] + df_socioecon['share_commute15'] + df_socioecon['share_commute20']
df_socioecon['medium_commute_25to45min'] = df_socioecon['share_commute25'] + df_socioecon['share_commute30'] + df_socioecon['share_commute35'] + df_socioecon['share_commute40']
df_socioecon['long_commute_45+min'] = df_socioecon['share_commute45'] + df_socioecon['share_commute60'] + df_socioecon['share_commute90']
df_socioecon['share_somehighereducation'] = df_socioecon['share_college'] + df_socioecon['share_associate'] + df_socioecon['share_professional'] + df_socioecon['share_bachelor'] + df_socioecon['share_master'] + df_socioecon['share_PhD']

In [ ]:
# Rename to _BG
for col in df_socioecon.columns:
    df_socioecon.rename(columns={col:col+'_byBG'},inplace=True)

In [ ]:
# Drop STATEFIPS_byBG
df_socioecon.drop('STATEFIPS_byBG',axis=1,inplace=True)

In [ ]:
# Save
df_socioecon.to_csv(result_path + 'CENSUS_' + selection)